# Construção de modelo de regressão dos dados de exportação do RN
## Experimento 1 - Pipeline de otimização com `MLFlow` e `AutoML`

<img height="200" src="https://cdn.mos.cms.futurecdn.net/VFLt5vHV7aCoLrLGjP9Qwm-1200-80.jpg" width="600"/>

In [1]:
import h2o
import pandas as pd
df = pd.read_csv("../../../../data/interim/preprocessing/04_dataset_expo_rn_OUTLIERS_featured_encoded.csv")
df

,Unnamed: 0,Bloco Econômico,Países,Código NCM,Descrição NCM,Código SH6,Descrição SH6,Código SH4,Descrição SH4,Código SH2,...,Descrição Seção,Unidade estatística,Valor US$ FOB,Quilograma Líquido,Quantidade Estatística,Data,Valor US$ FOB_mensal,Quilograma Líquido_mensal,Valor US$ FOB_anual,Quilograma Líquido_anual
0,0,0,8,328,1356,265,995,119,558,25,...,18,2,14906,11675,15,2024-09-01,571128,598314,453298187,783875921
1,1,0,37,618,1170,510,961,237,510,46,...,6,8,11130,1294,1294,2024-09-01,190282,25398,1552614,226207
2,2,0,37,215,188,170,917,66,452,16,...,16,8,38824,22782,22782,2024-09-01,1009459,570338,7618874,4239594
3,3,0,46,328,1356,265,995,119,558,25,...,18,2,1915,1482,2,2024-09-01,571128,598314,453298187,783875921
4,4,0,46,614,1464,506,1205,237,510,46,...,6,8,58126,9825,9825,2024-09-01,2769669,491933,11553520,2074862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57130,57130,6,161,213,471,169,429,66,452,16,...,16,8,9696,4640,0,1997-01-01,93303,63241,1990600,1261793
57131,57131,6,161,215,188,170,917,66,452,16,...,16,8,40640,30275,0,1997-01-01,294265,243061,6357902,5535543
57132,57132,6,161,216,1110,170,917,66,452,16,...,16,8,34060,26720,0,1997-01-01,294265,243061,6357902,5535543
57133,57133,7,75,205,213,161,207,61,111,14,...,2,8,45276,14000,14,1997-01-01,354955,84000,1463103,338500


In [2]:
df.describe()

,Unnamed: 0,Bloco Econômico,Países,Código NCM,Descrição NCM,Código SH6,Descrição SH6,Código SH4,Descrição SH4,Código SH2,...,Código Seção,Descrição Seção,Unidade estatística,Valor US$ FOB,Quilograma Líquido,Quantidade Estatística,Valor US$ FOB_mensal,Quilograma Líquido_mensal,Valor US$ FOB_anual,Quilograma Líquido_anual
count,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,57135.000000,...,57135.000000,57135.000000,57135.000000,5.713500e+04,5.713500e+04,5.713500e+04,5.713500e+04,5.713500e+04,5.713500e+04,5.713500e+04
mean,28567.000000,2.513853,72.001190,372.300674,701.468872,303.468364,612.923812,125.027759,336.222228,24.628039,...,4.790741,11.045961,7.660803,1.633967e+05,5.897617e+05,2.817190e+05,1.279993e+06,3.556204e+06,1.208358e+07,3.344291e+07
std,16493.598152,1.516221,46.735139,331.251800,449.492916,282.188113,359.285683,127.472742,147.585693,22.582640,...,4.821387,6.591323,1.255383,1.271327e+06,6.669498e+06,5.502427e+06,3.820337e+06,2.449288e+07,2.738699e+07,1.497925e+08
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,14283.500000,2.000000,31.000000,159.000000,350.000000,118.000000,398.000000,32.000000,232.500000,7.000000,...,1.000000,6.000000,8.000000,5.100000e+03,1.493500e+03,0.000000e+00,4.682050e+04,1.254400e+04,5.267520e+05,1.246600e+05
50%,28567.000000,2.000000,51.000000,213.000000,573.000000,169.000000,510.000000,66.000000,316.000000,16.000000,...,3.000000,16.000000,8.000000,2.364500e+04,1.294400e+04,1.440000e+03,3.474350e+05,1.383370e+05,4.346184e+06,1.665904e+06
75%,42850.500000,3.000000,117.000000,615.000000,1110.000000,507.000000,917.000000,237.000000,453.000000,46.000000,...,10.000000,17.000000,8.000000,8.637450e+04,4.454300e+04,1.809800e+04,9.789490e+05,7.438390e+05,1.120362e+07,9.192834e+06
max,57134.000000,7.000000,163.000000,1596.000000,1584.000000,1327.000000,1323.000000,559.000000,559.000000,88.000000,...,20.000000,20.000000,9.000000,1.154291e+08,6.116450e+08,6.341000e+08,1.293821e+08,1.064815e+09,4.532982e+08,2.296252e+09


In [ ]:
from h2o.automl import H2OAutoML

# Start the H2O cluster (with all available CPUs)
h2o.init(nthreads=-1)

# Import the dataset
data = h2o.import_file("../../../../data/interim/preprocessing/04_dataset_expo_rn_OUTLIERS_featured_encoded.csv")

# Split the dataset into training and testing sets (80% train, 20% test)
train, test = data.split_frame(ratios=[0.8], seed=1)

# Identify predictors and response
x = train.columns
y = "Valor US$ FOB_anual"
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models
horas_de_execucao = 15
minutos_execucao = horas_de_execucao * 60
segundos_execucao = minutos_execucao * 60

aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=segundos_execucao)
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.411-b09, mixed mode, sharing)
  Starting server from C:\Users\joao1\DataspellProjects\analise_exportacoes_rn\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\joao1\AppData\Local\Temp\tmpnaj77h47
  JVM stdout: C:\Users\joao1\AppData\Local\Temp\tmpnaj77h47\h2o_joao1_started_from_python.out
  JVM stderr: C:\Users\joao1\AppData\Local\Temp\tmpnaj77h47\h2o_joao1_started_from_python.err


C:\Users\joao1\DataspellProjects\analise_exportacoes_rn\.venv\Lib\site-packages\h2o\backend\server.py:395: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_joao1_70ydlu
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,247.5 Mb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


# Decision three com gridsearchCV

In [7]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Importar o dataset
data = pd.read_csv("../../../../data/interim/preprocessing/04_dataset_expo_rn_OUTLIERS_featured_encoded.csv")

# Identificar preditores e resposta
x = data.drop(columns=["Valor US$ FOB_anual"])  # Remover a coluna de resposta
y = data["Valor US$ FOB_anual"]

# Dividir o dataset em treinamento (80%) e teste (20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Definir o modelo
regressor = DecisionTreeRegressor(random_state=1)

# Definir o grid de hiperparâmetros
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['squared_error', 'friedman_mse']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5, verbose=1, n_jobs=-1)

# Realizar a busca
grid_search.fit(x_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Avaliação do modelo otimizado no conjunto de teste
y_pred = best_model.predict(x_test)

print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 120 candidates, totalling 600 fits
Melhores Hiperparâmetros: {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Mean Absolute Error (MAE): 56357.617193197395
Mean Squared Error (MSE): 355644744714.1052
R² Score: 0.9995904239711461


# Separando base de 1997 a 2022 como treino para tentar prever o restante.

> Árvore de decisão

In [15]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Importar o dataset
data = pd.read_csv("../../../../data/interim/preprocessing/04_dataset_expo_rn_OUTLIERS_featured_encoded.csv")

# Converter a coluna 'Data' para o formato datetime
data['Data'] = pd.to_datetime(data['Data'])

# Criar uma coluna 'Ano' para facilitar a separação
data['Ano'] = data['Data'].dt.year

# Separar os dados por ano (treino: até 2022, teste: 2023-2024)
train_data = data[data['Ano'] <= 2022]
test_data = data[data['Ano'] > 2022]

# Identificar preditores e resposta
x_train = train_data.drop(columns=["Valor US$ FOB_mensal", "Data"])
y_train = train_data["Valor US$ FOB_anual"]

x_test = test_data.drop(columns=["Valor US$ FOB_mensal", "Data"])
y_test = test_data["Valor US$ FOB_anual"]

# Definir o modelo
regressor = DecisionTreeRegressor(random_state=1)

# Definir o grid de hiperparâmetros
param_grid = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['squared_error', 'friedman_mse']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5, verbose=1, n_jobs=-1)

# Realizar a busca
grid_search.fit(x_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Avaliação do modelo otimizado no conjunto de teste
y_pred = best_model.predict(x_test)

print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 120 candidates, totalling 600 fits
Melhores Hiperparâmetros: {'criterion': 'friedman_mse', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mean Absolute Error (MAE): 1827643.4124917742
Mean Squared Error (MSE): 159091683148507.75
R² Score: 0.9583499035690904


# Separando base de 1997 a 2022 como treino para tentar prever o restante.

> Random Forest

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Importar o dataset
data = pd.read_csv("../../../../data/interim/preprocessing/04_dataset_expo_rn_OUTLIERS_featured_encoded.csv")

# Converter a coluna 'Data' para o formato datetime
data['Data'] = pd.to_datetime(data['Data'])

# Criar uma coluna 'Ano' para facilitar a separação
data['Ano'] = data['Data'].dt.year

# Separar os dados por ano (treino: até 2022, teste: 2023-2024)
train_data = data[data['Ano'] <= 2022]
test_data = data[data['Ano'] > 2022]

# Identificar preditores e resposta
x_train = train_data.drop(columns=["Valor US$ FOB_anual", "Data"])
y_train = train_data["Valor US$ FOB_anual"]

x_test = test_data.drop(columns=["Valor US$ FOB_anual", "Data"])
y_test = test_data["Valor US$ FOB_anual"]

# Definir o modelo (RandomForestRegressor)
regressor = RandomForestRegressor(random_state=1)

# Definir o grid de hiperparâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['squared_error', 'absolute_error']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           scoring='neg_mean_absolute_error', cv=5, verbose=1, n_jobs=-1)

# Realizar a busca
grid_search.fit(x_train, y_train)

# Melhor modelo encontrado
best_model = grid_search.best_estimator_

# Avaliação do modelo otimizado no conjunto de teste
y_pred = best_model.predict(x_test)

print("Melhores Hiperparâmetros:", grid_search.best_params_)
print("Mean Absolute Error (MAE):", mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


KeyboardInterrupt: 